# Set-up BW project

In [12]:
%run _imports.ipynb

## Create/open BW project and import databases

In [13]:
BW_PROJECT = 'iri_work' # insert your project name here
bw.projects.set_current(BW_PROJECT)

EI_DB = 'ecoinvent 3.9.1 cutoff' # name of ecoinvent database in your project
METALS_DB = "EV battery metals"

Import biosphere database

In [3]:
if 'biosphere3' in bw.databases:
    print('Biosphere database already installed')
else:
    bw.bw2setup()

Biosphere database already installed


Import ecoinvent database

*Note: The results presented in the scientific article were produced with ecoinvent 3.9.1 cut-off system model*

In [4]:
EI_PATH = 'XXXXXXXXX' # <- insert the path to ecoinvent datasets here

if EI_DB in bw.databases:
    print('Ecoinvent database already installed')
else:
    ei = bw.SingleOutputEcospold2Importer(EI_PATH, EI_DB)
    ei.apply_strategies()
    ei.statistics()
    ei.write_database()

Ecoinvent database already installed


## Import metals inventories

Import baseline inventories and link to ecoinvent and biosphere databases

In [14]:
lci_metals = bw.ExcelImporter(INVENTORIES_PATH)
lci_metals.apply_strategies()
lci_metals.match_database(EI_DB, fields=('name', 'reference product', 'unit', 'location'))
lci_metals.match_database("biosphere3", fields=('name', 'unit', 'categories'))

# The ExcelImporter requires that a 'reference product' is defined for every technosphere exchange.
# However, the ecoinvent database imported in BW2 as well as wurst use 'product' for technosphere exchanges.
# The following function replaces 'reference product' by 'product' in technosphere exchanges.
supporting_functions.correct_product_in_exchanges(lci_metals)

Extracted 5 worksheets in 0.61 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 9.30 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Create regionalized inventories for heating activities

In [4]:
# Import ecoinvent database into wurst format (i.e., list of dictionary, each dict being a dataset)
try:
  len(ei_db)
except NameError:
  ei_db = wurst.extract_brightway2_databases(EI_DB)

Getting activity data


100%|██████████| 21238/21238 [00:00<00:00, 76141.70it/s]


Adding exchange data to activities


100%|██████████| 674593/674593 [01:28<00:00, 7664.25it/s] 


Filling out exchange data


100%|██████████| 21238/21238 [00:03<00:00, 6143.68it/s] 


In [15]:
# Get inventories info:
breakdown_lists = supporting_functions.get_breakdown_lists()
technosphere = lambda x: x["type"] == "technosphere"

lci_regionalized = []

for ds in lci_metals.data:
    if ds["location"] == "GLO":
        pass
    else:
        for exc in filter(technosphere, ds["exchanges"]):
            if exc["product"] in breakdown_lists["heating products"] + ["coal tar"]:

                replicated_lcis = [(e["name"], e["reference product"], e["location"]) for e in lci_regionalized] 
                
                if (exc["name"], exc["product"], ds["location"]) in replicated_lcis:
                    ds_loc = [e for e in lci_regionalized if
                            e["name"] == exc["name"] and
                            e["reference product"] == exc["product"] and
                            e["location"] == ds["location"]][0]
                else:
                    exc_ds = wurst.get_one(
                        ei_db, 
                        wurst.contains("name", exc["name"]),
                        wurst.contains("reference product", exc["product"]),
                        wurst.contains("location", exc["location"])
                    )

                    ds_loc = supporting_functions.replicate_activity_to_location(exc_ds, ds["location"], ei_db, METALS_DB)
                    lci_regionalized.append(ds_loc)


                exc.update(
                        {"location": ds_loc["location"],
                        "database": ds_loc["database"],
                        "input": (ds_loc["database"], ds_loc["code"])
                        }
                    )

In [16]:
for ds in lci_regionalized:
    if ds["name"] == "steam production, as energy carrier, in chemical industry":
        for exc in filter(technosphere, ds["exchanges"]):
            if "heat" in exc["name"]:
                if (exc["name"], exc["product"], ds["location"]) in replicated_lcis:
                    ds_loc = [e for e in lci_regionalized if
                                e["name"] == exc["name"] and
                                e["reference product"] == exc["product"] and
                                e["location"] == ds["location"]][0]
                else:
                    exc_ds = wurst.get_one(
                            ei_db, 
                            wurst.contains("name", exc["name"]),
                            wurst.contains("reference product", exc["product"]),
                            wurst.contains("location", exc["location"])
                        )

                    ds_loc = supporting_functions.replicate_activity_to_location(exc_ds, ds["location"], ei_db, METALS_DB)
                    lci_regionalized.append(ds_loc)   
                    
                exc.update(
                            {"location": ds_loc["location"],
                            "database": ds_loc["database"],
                            "input": (ds_loc["database"], ds_loc["code"])
                            }
                        )             

## Create a new LCI database with these inventories

In [17]:
if METALS_DB in bw.databases:
    del bw.databases[METALS_DB]
wurst.write_brightway2_database(lci_metals.data + lci_regionalized, METALS_DB)

Writing activities to SQLite3 database:


28 datasets
621 exchanges
0 unlinked exchanges
  


0% [############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/06/2024 23:11:22
  Finished: 02/06/2024 23:11:23
  Total time elapsed: 00:00:00
  CPU %: 100.20
  Memory %: 32.06
Created database: EV battery metals
